# parse lido file and extract elements

In [1]:
import bs4 #beautifulSoup
import os
import re
from pprint import pprint
import pandas as pd

# Mapping Lido to imageSearch database fields

Need to map lido to the fields in our database.  

our db metadata fields are:

```
id | record_id | created_date | title | image_url | record_url | inventory_number | person | date | classification | material_technique | institution_isil | image_licence | year_min | year_max | classification_id_id | institution_isil_id_id
```

These correspond to the following Lido elements


`record_id`

```html
<lido:recordID lido:type=http://terminology.lido-schema.org/lido00100 lido:source=https://culture.ld.admin.ch/isil/CH-000511-9>21566</lido:recordID>
```


`title`

```html
<lido:titleWrap>
    <lido:titleSet>
        <lido:appellationValue>Porträt von Albrecht Dürer dem Älteren</lido:appellationValue>
    </lido:titleSet>
</lido:titleWrap>
```

`record_url`

```html
<lido:recordInfoLink>https://doi.org/10.16903/ethz-grs-D_008883</lido:recordInfoLink>
```

`person`

```html
<lido:displayActorInRole>Hollar, Wenzel (1607 - 1677)</lido:displayActorInRole>
 
<lido:displayActorInRole>Dürer, Albrecht (1471 - 1528), nach</lido:displayActorInRole>
```



`imageUrl`

```html
<lido:resourceRepresentation lido:type=http://terminology.lido-schema.org/lido00451>
<lido:linkResource>https://e-gs.ethz.ch/eMP/eMuseumPlus?service=ImageAsset&amp;module=collection&amp;objectId=21566&amp;resolution=mediumImageResolution</lido:linkResource>
</lido:resourceRepresentation>
```

`year_min`  `year_max`


```html
<lido:eventDate>
    <lido:displayDate>1498</lido:displayDate>
    <lido:date>
        <lido:earliestDate>1498</lido:earliestDate>
        <lido:latestDate>1498</lido:latestDate>
    </lido:date>
</lido:eventDate>
```

`classification`

ETH Zurich Graphische Sammlung

```html
<lido:classification lido:type="Objektklassifikation">
        <lido:conceptID lido:type=http://terminology.lido-schema.org/lido00099 lido:source=http://vocab.getty.edu/aat>
            http://vocab.getty.edu/aat/300041273
        </lido:conceptID>
    <lido:term>Druckgraphik</lido:term>
</lido:classification>
```

Zentral Bibliothek Graphische Sammlung

```html

    <lido:objectClassificationWrap>
        <lido:objectWorkTypeWrap>
            <lido:objectWorkType>
                <lido:term>Einblattdruck</lido:term>
            </lido:objectWorkType>
            <lido:objectWorkType>
                <lido:term>Bildliche Darstellung</lido:term>
            </lido:objectWorkType>
            <lido:objectWorkType>
                <lido:term>Holzschnitt</lido:term>
            </lido:objectWorkType>
        </lido:objectWorkTypeWrap>
    </lido:objectClassificationWrap>
```



`image_licence`

```html
<lido:rightsResource>
<lido:rightsType>
<lido:conceptID lido:type=http://terminology.lido-schema.org/lido00099>https://creativecommons.org/publicdomain/zero/1.0/</lido:conceptID>
<lido:term>CC0 1.0 Universal (CC0 1.0)</lido:term>
</lido:rightsType>
```

`material_technique`

```html
<lido:displayMaterialsTech>Radierung</lido:displayMaterialsTech>
 

<lido:termMaterialsTech lido:type=http://terminology.lido-schema.org/lido00132>
<lido:conceptID lido:type=http://terminology.lido-schema.org/lido00099 lido:source=http://vocab.getty.edu/aat>http://vocab.getty.edu/aat/300053241</lido:conceptID>
<lido:term>Radierung</lido:term>
</lido:termMaterialsTech>
```


## functions

In [2]:
def get_titles(object_identification_wrap: bs4.element.Tag):
    """
    find list of titles for a given object
    """
    
    titles = []
    for title_set in object_identification_wrap.find_all("lido:titleSet"):
        print(title_set.attrs)
        title = title_set.find("lido:appellationValue")
        if title:
            titles.append(title.text)
            
    return titles

In [3]:
def find_lido_element_text(lido: bs4.element.Tag, 
                            element_name:str) -> str:
    
    """
    generic function to a find an element from the given tag and return the innerText from the first found result
    """
    
    try:
        text = lido.find(element_name).text

    except Exception as E:
        print(E)
        text = None
    
    return text  
    

In [4]:
def get_lido_id(lido: bs4.element.Tag):
    """
    <lido:lidoRecID lido:type="http://terminology.lido-schema.org/lido00100" lido:source="http://ld.zdb-services.de/resource/organisations/DE-MUS-079214">DE-MUS-079214/lido/05091715,T,001</lido:lidoRecID>
    """ 

    try:
        _id = lido.find("lido:lidoRecID").text
    except Exception as E:
        print(E)
        _id = None
    
    return _id

In [5]:
def get_image_url(lido: bs4.element.Tag) -> str:
    """
    assume there is only one image and that we only use lido:type 'http://terminology.lido-schema.org/lido00451'
    
    return the url for the image or None if element is not found 
    """
    
    rep_types_to_use = [
        "http://terminology.lido-schema.org/lido00451",
        "http://terminology.lido-schema.org/resourceRepresentation_type/provided_image",
    ]
    
    resource_reps = lido.find_all("lido:resourceRepresentation")
    
    for resource_rep in resource_reps:
        rep_type = resource_rep.get("lido:type")
        if rep_type in rep_types_to_use:
            url = resource_rep.find("lido:linkResource")
            if url:
                return url.text

    return None

In [6]:
def find_record_id(lido: bs4.element.Tag) -> str:
    
    res = lido.find("lido:lidoRecID", attrs = {"lido:type":"http://terminology.lido-schema.org/identifier_type/local_identifier"})
    if res:
        return res.text
    
    res = lido.find("lido:recordID", attrs={"lido:type":"http://terminology.lido-schema.org/lido00100"})
    if res:
        return res.text
    
    return None

In [7]:
def find_record_url(lido: bs4.element.Tag) -> str:
    
    res = lido.find("lido:recordInfoLink")

    if res:
        return res.text            
    return None

In [8]:
def find_inventory_number(lido: bs4.element.Tag) -> str:
    
    res = lido.find("lido:workID", attrs={"lido:type":"Inventarnummer"})

    if res:
        return res.text            
    return None


In [9]:
def find_persons(lido) -> [str,]:
    
    res = lido.find_all("lido:displayActorInRole")
    
    if res:
        return [r.text for r in res]

    return None

In [10]:

def find_relevant_dates(lido) -> [str,]:
    
    # lido:term xml:lang="en">Publication event
    # http://terminology.lido-schema.org/identifier_type/uri">http://terminology.lido-schema.org/lido00485
    
    event_types_to_find= [
    "http://terminology.lido-schema.org/lido00485", # publication date
    "http://terminology.lido-schema.org/lido00486",
    "http://terminology.lido-schema.org/lido00484",
    "http://terminology.lido-schema.org/lido00487",
    "http://terminology.lido-schema.org/lido00228",
    ]
    found_dates=[]

    events = lido.find_all("lido:event")
    for event in events:
        event_type = event.find("lido:eventType").find("lido:conceptID").text # attrs={"lido:type":"http://terminology.lido-schema.org/lido00099"}
        if event_type in event_types_to_find:
            event_dates = find_min_max_years_within_event(event)
            found_dates +=event_dates

    return found_dates

In [11]:
def find_min_max_years_within_event(lido) -> (int, int):
    """
    find all listed dates and take min max
    
    """
    
    found_date_strings = []
    date_types = ["lido:displayDate", "lido:earliestDate", "lido:latestDate",]
    
    for date_type in date_types:
        dates = lido.find_all(date_type)
        for date in dates:
            found_date_strings.append(date.text)
                         
    return found_date_strings


In [12]:
def find_years_in_string(s: str) -> list:
    """
    finds 4 digit numbers in strings
    e.g. for input "earliest year is 1591 - to (1700)"
    returns [1591, 1700]
    """

    ptrn = r"[0-9]{4}"
    res = re.findall(ptrn, s)
    return res

In [13]:
def get_min_max_year_from_dates(date_strings:[str,]) -> (int, int):
    
    years = []
    for s in date_strings:
        years += find_years_in_string(s)

    if years:
        year_min = min(years)
        year_max = max(years)
    else:
        year_min = None
        year_max = None

    return year_min, year_max

In [14]:
def find_title(lido: bs4.element.Tag) -> str:
    """
    can be multiple titles. this function just finds the first
    """
    try:
        title_element = lido.find("lido:titleSet").find("lido:appellationValue")
    except Exception as e:
        return None
    
    return title_element.text

In [15]:
def find_classification(lido: bs4.element.Tag) -> str:

    cls = lido.find("lido:classification")
    
    if cls:
        return cls.find("lido:term").text
    
    return None

In [16]:
def find_classification_2(lido: bs4.element.Tag) -> str:
    """


    `
        <lido:objectClassificationWrap>
            <lido:objectWorkTypeWrap>
                <lido:objectWorkType>
                    <lido:term>Einblattdruck</lido:term>
                </lido:objectWorkType>
                <lido:objectWorkType>
                    <lido:term>Bildliche Darstellung</lido:term>
                </lido:objectWorkType>
                <lido:objectWorkType>
                    <lido:term>Holzschnitt</lido:term>
                </lido:objectWorkType>
            </lido:objectWorkTypeWrap>
        </lido:objectClassificationWrap>
    `
    """
    class_el =  lido.find("lido:objectClassificationWrap")
    if class_el:
        
        classes = class_el.find_all("lido:objectWorkType")
        return [cls.findChildren()[0].text for cls in classes]
    
    return None

In [17]:
def find_insitution_isil(lido)->str:

    isil = lido.find("lido:legalBodyID", attrs={"lido:type":"http://terminology.lido-schema.org/lido00099"})

    if isil:
        return isil.text
    
    isil = lido.find("lido:legalBodyID", attrs={"lido:type":"http://terminology.lido-schema.org/identifier_type/uri"})
    if isil:
        return isil.text
    
    return None

def find_insitution_name(lido)->str:

    el = lido.find("lido:legalBodyName")
    
    if el:
        return el.find("lido:appellationValue").text

    return None


In [18]:
def find_image_licence(lido) -> str:
    
    el = lido.find("lido:rightsResource")
    if el:
        el = el.find("lido:term")

    if el:
        return el.text
    
    return None

In [19]:
def find_credit_line(lido) -> str:
    
    el = lido.find("lido:creditLine")
    
    if el:
        return el.text
    
    return None

In [20]:
def find_material_techniques(lido) -> [str,]:
    
    mat_tecs = lido.find_all("lido:displayMaterialsTech")
    
    if mat_tecs:
        return [mat_tec.text for mat_tec in mat_tecs]

    return None

In [21]:
def find_all_lido_mappings(lido) -> dict:
    
    lido_dict = {}
    # find record_id
    lido_dict['record_id'] = find_record_id(lido)
    
    # find publisher_id
    lido_dict['object_published_id'] = find_lido_element_text(lido,"lido:objectPublishedID")
    
    # find lido_id
    lido_dict['lido_id'] = get_lido_id(lido)
    
    # find image_url
    lido_dict['image_url'] = get_image_url(lido)
    
    # find record_url
    lido_dict['record_url'] = find_record_url(lido)
    
    # find inventory_number
    lido_dict['inventory_number'] = find_inventory_number(lido)
    
    # find host_isil
    lido_dict['institution_isil'] = find_insitution_isil(lido)
    lido_dict['institution_name'] = find_insitution_name(lido)
    
    # find image_licence
    lido_dict['image_licence'] = find_image_licence(lido)
    
    # find creditline
    lido_dict['credit_line'] = find_credit_line(lido)
    
    # find title
    lido_dict['title'] = find_title(lido)
    
    # find person(s)
    lido_dict['person'] = find_persons(lido)    
    
    # find material_technique
    lido_dict['material_technique'] = find_material_techniques(lido)
    
    # find min max years
    found_date_strings = find_relevant_dates(lido)
    year_min, year_max = get_min_max_year_from_dates(found_date_strings)
    lido_dict['year_min'] = year_min
    lido_dict['year_max'] = year_max

    # find classification
    lido_dict['classification'] = find_classification(lido)
    if lido_dict['classification']==None:
        lido_dict['classification'] = find_classification_2(lido)
    
    return lido_dict

# MAIN

In [22]:
# set up filepaths
institution="zbz"
fname = "DL-ZB-20201209-20210219.LIDO.xml"
fpath=None

In [25]:
DATA_DIR = "../../data/"
dir_path = f"raw/{institution}/"

flist = os.listdir(DATA_DIR + dir_path)
print("available files: ", [f for f in flist if f.endswith(".xml")])

if fpath==None:
    fpath = os.path.join(DATA_DIR, dir_path, fname)
    
print(f"opening file: {fpath}")

available files:  ['DL-ZB-20201209-20210219.LIDO.xml']
opening file: ../../data/raw/zbz/DL-ZB-20201209-20210219.LIDO.xml


In [26]:
# load lido file
with open(fpath, 'r') as f:
    content = f.read()

print("parsing with BeautifulSoup ...") 
bs_content = bs4.BeautifulSoup(content, "xml")

parsing with BeautifulSoup ...


In [56]:
# find all the individual lido records, as the main element that contains all the metadata for an object
lidos = bs_content.find_all('lido:lido')
print(f"found lido elements: {len(lidos)}")

found lido elements: 27935


In [57]:
# prepare output file to save to
output_dir_path = DATA_DIR + f"interim/{institution}/metadata/"

if os.path.exists(output_dir_path)==False:
    os.makedirs(output_dir_path)

output_fpath = output_dir_path + "metadata.csv"
output_fpath

'../../data/interim/zbz/metadata/metadata.csv'

In [58]:
chunksize=100
num_records = len(lidos)

lido_dict_list = []
for i, lido in enumerate(lidos):

    lido_dict = find_all_lido_mappings(lido)
    lido_dict_list.append(lido_dict)
    
    # add record to list / write to db
    # save at each chunk interval or on last record
    if (len(lido_dict_list) >= chunksize) or (i+1==num_records):
        # write to csv
        tdf = pd.DataFrame(lido_dict_list)
        try: 
            tdf.to_csv(output_fpath, mode="a")
        except FileNotFoundError as e:            
            tdf.to_csv(output_fpath, mode="w")           
        # clear list
        lido_dict_list = []
        print(f"\rprocessed {i+1} of {num_records}", end="")
print(":  done processing lido file")

processed 27934 of 27935